## This notebook demonstrates bundles to seqex example

Install packages required for apache-beam and protobuf

In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from absl import app
from absl import flags
import apache_beam as beam
from google.protobuf import text_format
from proto.stu3 import google_extensions_pb2
from proto.stu3 import resources_pb2
from proto.stu3 import version_config_pb2
from tensorflow.core.example import example_pb2
from py.google.fhir.seqex import bundle_to_seqex
import time

In [2]:
def _get_version_config(version_config_path):
  with open(version_config_path) as f:
    return text_format.Parse(f.read(), version_config_pb2.VersionConfig())

# Initialize variables

In [10]:
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter

import apache_beam as beam
import re

options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'de-test-224618'
google_cloud_options.job_name = 'job1'
google_cloud_options.staging_location = 'gs://healthedatalab/staging'
google_cloud_options.temp_location = 'gs://healthedatalab/temp'
options.view_as(StandardOptions).runner = 'DirectRunner'


In [11]:
input_path = 'gs://healthedatalab/synthea/bundles/bundles*'
label_path = 'gs://healthedatalab/synthea/labels/label*'
output_path = 'gs://healthedatalab/synthea/seqex/seqex'

## Transform definition

In [7]:
p = beam.Pipeline(options=options)
version_config = _get_version_config("/usr/local/fhir/proto/stu3/version_config.textproto")

keyed_bundles = ( 
    p 
    | 'readBundles' >> beam.io.ReadFromTFRecord(
        input_path, coder=beam.coders.ProtoCoder(resources_pb2.Bundle))
    | 'KeyBundlesByPatientId' >> beam.ParDo(
        bundle_to_seqex.KeyBundleByPatientIdFn()))
event_labels = ( 
    p | 'readEventLabels' >> beam.io.ReadFromTFRecord(
        label_path,
        coder=beam.coders.ProtoCoder(google_extensions_pb2.EventLabel)))
keyed_event_labels = bundle_to_seqex.CreateTriggerLabelsPairLists(
    event_labels)
bundles_and_labels = bundle_to_seqex.CreateBundleAndLabels(
    keyed_bundles, keyed_event_labels)
_ = ( 
    bundles_and_labels
    | 'Reshuffle1' >> beam.Reshuffle()
    | 'GenerateSeqex' >> beam.ParDo(
        bundle_to_seqex.BundleAndLabelsToSeqexDoFn(
            version_config=version_config,
            enable_attribution=False,
            generate_sequence_label=False))
    | 'Reshuffle2' >> beam.Reshuffle()
    | 'WriteSeqex' >> beam.io.WriteToTFRecord(
        output_path,
        coder=beam.coders.ProtoCoder(example_pb2.SequenceExample),
        file_name_suffix='.tfrecords',
        num_shards=2))

I0115 21:59:49.419306 140485840406272 gcsio.py:446] Starting the size estimation of the input
I0115 21:59:49.422370 140485840406272 client.py:614] Attempting refresh to obtain initial access_token
I0115 21:59:49.517318 140485840406272 gcsio.py:460] Finished listing 9 files in 0.0980060100555 seconds.
I0115 21:59:49.530730 140485840406272 gcsio.py:446] Starting the size estimation of the input
I0115 21:59:49.534348 140485840406272 client.py:614] Attempting refresh to obtain initial access_token
I0115 21:59:49.625538 140485840406272 gcsio.py:460] Finished listing 1 files in 0.0948059558868 seconds.


Run the transform

In [12]:
start = time.time()
p.run().wait_until_finish()
end = time.time()
print(end-start)

I0115 22:20:23.784215 140485840406272 fn_api_runner.py:844] ==================== <function annotate_downstream_side_inputs at 0x7fc519392f50> ====================
I0115 22:20:23.788387 140485840406272 fn_api_runner.py:844] ==================== <function fix_side_input_pcoll_coders at 0x7fc519392230> ====================
I0115 22:20:23.792687 140485840406272 fn_api_runner.py:844] ==================== <function lift_combiners at 0x7fc519392668> ====================
I0115 22:20:23.795468 140485840406272 fn_api_runner.py:844] ==================== <function expand_gbk at 0x7fc519392938> ====================
I0115 22:20:23.815741 140485840406272 fn_api_runner.py:844] ==================== <function sink_flattens at 0x7fc5193925f0> ====================
I0115 22:20:23.818279 140485840406272 fn_api_runner.py:844] ==================== <function greedily_fuse at 0x7fc5193926e0> ====================
I0115 22:20:23.823961 140485840406272 fn_api_runner.py:844] ==================== <function sort_stag

I0115 22:21:02.856411 140485840406272 bundle_processor.py:303] finish <DoOperation CreateTriggerLabelsPairLists output_tags=['out'], receivers=[ConsumerSet[CreateTriggerLabelsPairLists.out0, coder=WindowedValueCoder[TupleCoder[BytesCoder, FastPrimitivesCoder]], len(consumers)=1]]>
I0115 22:21:02.858251 140485840406272 bundle_processor.py:303] finish <DoOperation GroupBundleAndTriggers/pair_with_trigger_labels_pair_lists output_tags=['out'], receivers=[ConsumerSet[GroupBundleAndTriggers/pair_with_trigger_labels_pair_lists.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], LengthPrefixCoder[FastPrimitivesCoder]]], len(consumers)=1]]>
I0115 22:21:02.860325 140485840406272 bundle_processor.py:303] finish <FlattenOperation GroupBundleAndTriggers/Flatten/Transcode/0 receivers=[ConsumerSet[GroupBundleAndTriggers/Flatten/Transcode/0.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], LengthPrefixCoder[FastPrimitivesCoder]]], len(cons

I0115 22:24:31.204670 140485840406272 bundle_processor.py:303] finish <DoOperation GenerateSeqex output_tags=['out'], receivers=[ConsumerSet[GenerateSeqex.out0, coder=WindowedValueCoder[ProtoCoder], len(consumers)=1]]>
I0115 22:24:31.206506 140485840406272 bundle_processor.py:303] finish <DoOperation Reshuffle2/AddRandomKeys output_tags=['out'], receivers=[ConsumerSet[Reshuffle2/AddRandomKeys.out0, coder=WindowedValueCoder[TupleCoder[FastPrimitivesCoder, ProtoCoder]], len(consumers)=1]]>
I0115 22:24:31.210464 140485840406272 bundle_processor.py:303] finish <DoOperation Reshuffle2/ReshufflePerKey/Map(reify_timestamps) output_tags=['out'], receivers=[ConsumerSet[Reshuffle2/ReshufflePerKey/Map(reify_timestamps).out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], TupleCoder[LengthPrefixCoder[ProtoCoder], LengthPrefixCoder[FastPrimitivesCoder]]]], len(consumers)=1]]>
I0115 22:24:31.213207 140485840406272 bundle_processor.py:303] finish <DataOutputOperation Resh

I0115 22:26:01.374177 140485840406272 fn_api_runner.py:912] Running ((ref_PCollection_PCollection_32/Read)+(ref_AppliedPTransform_WriteSeqex/Write/WriteImpl/PreFinalize_57))+(ref_PCollection_PCollection_40/Write)
I0115 22:26:01.467401 140485840406272 bundle_processor.py:291] start <DataOutputOperation ref_PCollection_PCollection_40/Write >
I0115 22:26:01.468976 140485840406272 bundle_processor.py:291] start <DoOperation WriteSeqex/Write/WriteImpl/PreFinalize output_tags=['out']>
I0115 22:26:01.471606 140485840406272 bundle_processor.py:291] start <DataInputOperation ref_PCollection_PCollection_32/Read receivers=[ConsumerSet[ref_PCollection_PCollection_32/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
I0115 22:26:01.476310 140485840406272 gcsio.py:446] Starting the size estimation of the input
I0115 22:26:01.479831 140485840406272 client.py:614] Attempting refresh to obtain initial access_token
I0115 22:26:01.570666 140485840406272 gcsio.py:460] Finished l

340.844485998


YAY!!